In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [2]:
def init_browser():
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
url = "https://mars.nasa.gov/news"
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_twtr = "https://twitter.com/MarsWxReport?lang=en"
url_table = "https://space-facts.com/mars/"
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [4]:
browser = init_browser()
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
#news_text = soup.find('div', class_='list_text')
#new_parsing = news_text.find("div", class_="content_title")
news_title = soup.find('a', target="_self").text

In [5]:
browser = init_browser()
browser.visit(url_img)
browser.click_link_by_partial_text("FULL IMAGE")
browser.click_link_by_partial_text("more info")
html = browser.html
soup = bs(html, "html.parser")
mars_img = soup.find('figure', class_="lede").a["href"]
mars_img = f"https://www.jpl.nasa.gov{mars_img}"

C:\Users\jacob\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [6]:
browser = init_browser()
browser.visit(url_twtr)
html = browser.html
soup = bs(html, "html.parser")
mars_weather = soup.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")
mars_weather

In [7]:
mars_table_from_html = pd.read_html(url_table)
dataframe = mars_table_from_html[0]
mars_table = dataframe.rename(columns = {0 : "Category", 1 : "Fact"})
mars_table = mars_table.set_index("Category")

In [8]:
mars_table = mars_table.to_dict()
mars_table

{'Fact': {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.38 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-87 to -5 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'}}

In [9]:
browser = init_browser()
browser.visit(url_hemispheres)
html = browser.html
soup = bs(html, "html.parser")
hemi_list = soup.find('div', class_ = 'collapsible results')
mars_hemis = hemi_list.find_all('div', class_='item')
mars_hemis
hemi_imgs = []
for hemi in mars_hemis:
    i = hemi.find('img')['src']
    i = f"https://astrogeology.usgs.gov{i}"
    hemi_imgs.append(i)
hemi_img_1 = hemi_imgs[0]
hemi_img_2 = hemi_imgs[1]
hemi_img_3 = hemi_imgs[2]
hemi_img_4 = hemi_imgs[3]

In [10]:
mars_scrape = {"News_Title": news_title,
           "Mars_Image": mars_img,
           "Mars_Weather": mars_weather,
           "Mars_Table": mars_table,
           "Hemisphere_1": hemi_img_1,
           "Hemisphere_2": hemi_img_2,
           "Hemisphere_3": hemi_img_3,
           "Hemisphere_4": hemi_img_4}

In [11]:
mars_scrape

{'News_Title': 'The Red Planet',
 'Mars_Image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA12826_hires.jpg',
 'Mars_Weather': None,
 'Mars_Table': {'Fact': {'Equatorial Diameter:': '6,792 km',
   'Polar Diameter:': '6,752 km',
   'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
   'Moons:': '2 (Phobos & Deimos)',
   'Orbit Distance:': '227,943,824 km (1.38 AU)',
   'Orbit Period:': '687 days (1.9 years)',
   'Surface Temperature:': '-87 to -5 °C',
   'First Record:': '2nd millennium BC',
   'Recorded By:': 'Egyptian astronomers'}},
 'Hemisphere_1': 'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png',
 'Hemisphere_2': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png',
 'Hemisphere_3': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png',
 'Hemisphere_4': 'https://astrogeology.usgs.gov/cache/images/0

In [12]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
mars_db = client["mars_db"]
collection = mars_db["mars_data"]
insert = mars_db.collection.insert_one(mars_scrape)